# Create Data set

In [92]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes

In [93]:
from sklearn import set_config
set_config(display='diagram')

In [94]:
X, y = load_diabetes(return_X_y=True)

In [95]:
print(X.shape, y.shape)

(442, 10) (442,)


# Train test split

In [96]:
from sklearn.model_selection import train_test_split

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [98]:
print(X_train.shape, X_test.shape)

(353, 10) (89, 10)


In [99]:
print(y_train.shape, y_test.shape)

(353,) (89,)


# Apply SGD Regressor

In [100]:
from sklearn.linear_model import SGDRegressor

In [101]:
# give parameter 
sgd = SGDRegressor(
    penalty='l2', # ridge 
    max_iter=500, # epochs
    learning_rate= 'constant',
    alpha= 0.001, 
    eta0= 0.01, 
)

In [102]:
# Now give the data to learn this parameters

sgd.fit(X_train, y_train)

SGDRegressor(alpha=0.001, learning_rate='constant', max_iter=500)

In [103]:
# predict 
y_pred_sgd = sgd.predict(X_test)
y_pred_sgd.shape

(89,)

In [104]:
from sklearn.metrics import r2_score

In [105]:
# check r2 score
r2_score(y_test, y_pred_sgd)

0.43354692874334066

In [106]:
sgd.coef_

array([  56.36738683, -102.17588671,  320.62859837,  238.32079064,
         17.25219271,  -27.752187  , -161.38041918,  141.22055562,
        289.98952503,  115.36788264])

In [107]:
sgd.intercept_

array([149.00675695])

# Apply Ridge 

In [108]:
from sklearn.linear_model import Ridge

In [109]:
rr = Ridge(
    alpha=0.001,
    max_iter=500,
    solver='sparse_cg'
)

In [110]:
# now fit the data to learn this parameters
rr.fit(X_train, y_train)

Ridge(alpha=0.001, max_iter=500, solver='sparse_cg')

In [111]:
y_pred_rr = rr.predict(X_test)
y_pred_rr.shape

(89,)

In [112]:
# now check the r2 score
# we can see the improvement
r2_score(y_test, y_pred_rr)

0.46250101619914563

In [113]:
rr.coef_

array([  34.52192544, -290.84084076,  482.40181344,  368.0678662 ,
       -852.44873179,  501.59160336,  180.11115788,  270.76333979,
        759.73534372,   37.4913546 ])

In [114]:
rr.intercept_

np.float64(151.10198517439466)

# Make our own Model 

In [115]:
class RidgeGradientDescentRegressor:
    
    def __init__(self, epochs= 500, learning_rate=0.005, alpha=0.001):
        self.epochs = epochs
        self.lr = learning_rate
        self.alpha = alpha
        self.coef_ = None
        self.intercept_ = None
        
    def fit(self, X, y):
        # initialization 
        self.intercept_ = 0
        self.coef_ = np.ones(X.shape[1])
        X = np.insert(X, 0, 1, axis=1)
        theta = np.insert(self.coef_, 0, self.intercept_)
        
        # Calculate the derivative
        for i in range(self.epochs):
            theta_derivative = np.dot(X.T, X).dot(theta) - np.dot(X.T, y) + (self.alpha * theta)
            theta = theta - ( self.lr * theta_derivative)
            
        # Update the parameters
        self.coef_ = theta[1:]
        self.intercept_ = theta[0]
        
    def predict(self, X):
        return np.dot(X, self.coef_) + self.intercept_
        pass

In [116]:
ridge_own = RidgeGradientDescentRegressor(epochs=500, learning_rate=0.005, alpha=0.001)

In [117]:
ridge_own.fit(X_train, y_train)

In [118]:
ridge_own.coef_

array([  46.65050914, -221.3750037 ,  452.12080647,  325.54248128,
        -29.09464178,  -96.47517735, -190.90017011,  146.32900372,
        400.80267299,   95.09048094])

In [119]:
ridge_own.intercept_

np.float64(150.86975316713466)

In [120]:
y_pred_rr_own = ridge_own.predict(X_test)
y_pred_rr_own.shape

(89,)

In [121]:
# compare all three
print(f"r2 Score of sgd Linear model without ridge {r2_score(y_test, y_pred_sgd)} \n ")
print(f"r2 Score of applying ridge {r2_score(y_test, y_pred_rr)} \n ")
print(f"r2 Score of applying our own ridge model {r2_score(y_test, y_pred_rr_own)} \n ")

r2 Score of sgd Linear model without ridge 0.43354692874334066 
 
r2 Score of applying ridge 0.46250101619914563 
 
r2 Score of applying our own ridge model 0.4738018280260913 
 
